Este codigo se aplica solo a titulos en español 

In [2]:
import pandas as pd 
ruta_csv = 'C:/Users/Allis/OneDrive/UniversidadCenfo2025/Practicaclase2procesos/STEM_LADAINA/7_Datasets_Variaciones/Dataset_DivididoPorIdioma.csv'
df = pd.read_csv(ruta_csv)
print(df.head())

         Fuente            Categoría  \
0  Computrabajo  Software Developers   
1  Computrabajo  Software Developers   
2  Computrabajo  Software Developers   
3  Computrabajo       Web Developers   
4  Computrabajo       Web Developers   

                                              Título  \
0                             Analista desarrollador   
1                          Programador Sistemas .Net   
2  Técnico en Soporte Informático San José Cartag...   
3                   Desarrollador Senior TI (Oracle)   
4                     Técnico en Soporte informático   

                                             Empresa  \
0  FACTOR K QUINIENTOS SEIS SC SOCIEDAD DE RESPON...   
1                     Architect Technology Solutions   
2                                  SOPORTEXPERTO.COM   
3                                            Asembis   
4                                    No especificado   

                          Ubicación        Modalidad Idioma Solicitado  \
0      Here

In [ ]:
import re
import pandas as pd
import os
from collections import Counter

# === CARGAR DATASET ===


# ========================================
# 1️⃣ DICCIONARIO DE SENIORITY / NIVEL
# ========================================
seniority_dict = {
    "jr": "junior", "jr.": "junior", "junior": "junior", "joven": "junior",
    "ssr": "semisenior", "semi senior": "semisenior", "semi-senior": "semisenior",
    "sr": "senior", "sr.": "senior", "senior": "senior", "avanzado": "senior",
    "lead": "senior", "líder": "senior", "lider": "senior", "principal": "senior", 
    "experto": "senior", "manager": "senior", "coordinador": "senior", 
    "supervisor": "senior", "jefe": "senior", "gerente": "senior", 
    "director": "senior", "regente": "senior", "encargado": "senior",
    "responsable": "senior", "capitan": "senior", "gestor": "senior",
    "subgerente": "senior", "ejecutivo": "senior"
}

# ========================================
# 2️⃣ DICCIONARIO DE MODALIDAD / JORNADA
# ========================================
modalidad_dict = {
    "remoto": "remoto", "remote": "remoto", "teletrabajo": "remoto", "casa": "remoto",
    "híbrido": "hibrido", "hibrido": "hibrido", "hybrid": "hibrido",
    "presencial": "presencial", "onsite": "presencial", "oficina": "presencial",
    "full time": "tiempo completo", "jornada completa": "tiempo completo", 
    "tiempo completo": "tiempo completo", "completo": "tiempo completo",
    "part time": "medio tiempo", "medio tiempo": "medio tiempo", 
    "tiempo parcial": "medio tiempo", "parcial": "medio tiempo",
    "jornada diurna": "diurna", "diurno": "diurna",
    "jornada nocturna": "nocturna", "nocturno": "nocturna",
    "temporal": "temporal", "por horas": "temporal", "contrato": "temporal",
    "definido": "temporal", "freelance": "freelance", 
    "practicante": "intern", "internship": "intern", "pasante": "intern",
    "rotativo": "rotativo", "rotativa": "rotativo"
}

# ========================================
# 3️⃣ DICCIONARIO DE SINÓNIMOS DE PUESTOS
# ========================================
puesto_dict = {
    # Desarrollo de Software
    "desarrollador": "developer", "programador": "developer",
    "backend developer": "backend", "frontend developer": "frontend",
    "fullstack developer": "fullstack", "full stack": "fullstack",
    "software engineer": "ingeniero software", "ingeniero de software": "ingeniero software",
    "devops": "devops", "desarrollo": "developer",
    
    # Datos e IA
    "data engineer": "ingeniero datos", "ml engineer": "ingeniero ml",
    "ai engineer": "ingeniero ia", "analista": "analista",
    "data analyst": "analista datos", "analista de datos": "analista datos",
    "cientifico de datos": "data scientist", "data scientist": "data scientist",
    "bi": "business intelligence", "business intelligence": "business intelligence",
    
    # Soporte e Infraestructura
    "soporte técnico": "soporte", "técnico en soporte": "soporte",
    "soporte informático": "soporte", "soporte": "soporte",
    "network engineer": "ingeniero redes", "infraestructura": "infraestructura",
    "administrador": "administrador", "administrador de sistemas": "administrador sistemas",
    
    # Ingenierías
    "ingeniero": "ingeniero", "ingeniera": "ingeniero",
    "ingeniero civil": "ingeniero civil", "civil": "ingeniero civil",
    "ingeniero eléctrico": "ingeniero electrico", "electrico": "ingeniero electrico",
    "ingeniero electromecánico": "ingeniero electromecanico", "electromecanico": "ingeniero electromecanico",
    "ingeniero industrial": "ingeniero industrial", "industrial": "ingeniero industrial",
    "ingeniero mecánico": "ingeniero mecanico", "mecanico": "ingeniero mecanico",
    "ingeniero de calidad": "ingeniero calidad", "calidad": "ingeniero calidad",
    "ingeniero de proyectos": "ingeniero proyectos", "proyectos": "ingeniero proyectos",
    
    # Diseño
    "diseñador": "diseñador", "diseñadora": "diseñador",
    "ux designer": "diseñador ux", "ui designer": "diseñador ui",
    "ux/ui": "diseñador uxui", "diseñador gráfico": "diseñador grafico",
    
    # Gestión
    "project manager": "gerente proyecto", "scrum master": "scrum master",
    "coordinador de proyecto": "gerente proyecto", "product owner": "product owner", 
    "product manager": "product manager",
    
    # Técnicos
    "técnico": "tecnico", "tecnologo": "tecnico", "technician": "tecnico",
    "mecánico": "mecanico", "electricista": "electricista",
    
    # Comercial/Ventas
    "vendedor": "vendedor", "comercial": "vendedor", "sales": "vendedor",
    "ejecutivo": "ejecutivo", "asesor": "asesor",
    
    # Especializados
    "farmacéutico": "farmaceutico", "microbiólogo": "microbiologo",
    "agrónomo": "agronomo", "veterinario": "veterinario"
}

# ========================================
# 4️⃣ LISTA DE UBICACIONES (Costa Rica)
# ========================================
ubicaciones_cr = [
    "costa rica", "san jose", "alajuela", "cartago", "heredia",
    "guanacaste", "puntarenas", "limon", "limón",
    "curridabat", "desamparados", "escazu", "escazú", "moravia", "tibas", "tibás",
    "santa ana", "belen", "belén", "barva", "santo domingo", "naranjo",
    "palmares", "upala", "nicoya", "liberia", "bagaces", "carrillo",
    "esparza", "osa", "corredores", "pococi", "pococí", "siquirres", 
    "pozos", "goicoechea", "sabanilla", "coronado", "san pablo",
    "san rafael", "san isidro", "santa domingo", "la union", "la unión",
    "san sebastian", "san sebastián", "purral", "lagunilla", "san antonio",
    "san luis", "san vicente", "san miguel", "san francisco", "san juan",
    "san pedro", "san marcos", "san ramon", "san ramón"
]

# ========================================
# 5️⃣ PALABRAS DE RUIDO
# ========================================
palabras_ruido = [
    # Licencias y requisitos de transporte
    "licencia", "b1", "b2", "b3", "al dia", "al día", "vigente",
    "montacargas", "vehiculo propio", "vehículo propio", "moto", "automóvil",
    
    # Disponibilidad
    "disponibilidad inmediata", "disponibilidad para viajar", "disponibilidad",
    "giras", "trasladarse", "movilizarse",
    
    # Experiencia
    "experiencia comprobada", "experiencia deseable", "experiencia",
    "años de experiencia", "anos de experiencia", "minimo", "mínimo",
    
    # Horarios
    "jornada", "turno", "diurno", "nocturno", "rotativo", "horario",
    
    # Entornos laborales
    "produccion", "producción", "planta", "operario", "operativa",
    "empresa", "contrato", "indefinido", "temporal", "corporativo",
    
    # Verbos de solicitud
    "se requiere", "requerimos", "solicitamos", "buscamos", "necesitamos",
    
    # Educación
    "bachillerato", "universitario", "licenciatura",
    "estudiante", "graduado", "titulo", "título",
    
    # Ubicaciones y referencias espaciales
    "sector", "zona", "ubicado en", "ubicada en", "oficina", "supermercado", 
    "bodega", "tienda", "local", "centro", "alrededores",
    
    # Palabras vacías
    "a", "o", "en", "para", "de", "con", "y", "el", "la", "los", "las", 
    "urgente", "importante", "nuevo", "nueva", "iii", "i",
] + ubicaciones_cr 


# ========================================
# 6️⃣ FUNCIÓN DE LIMPIEZA TOTAL
# ========================================
def limpiar_titulo_total(titulo):
    if pd.isna(titulo):
        return ""
    s = str(titulo).lower().strip()
    
    # Normalizar tildes
    s = (s.replace("á", "a").replace("é", "e")
           .replace("í", "i").replace("ó", "o")
           .replace("ú", "u").replace("ñ", "n"))
    
    # Eliminar símbolos raros
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    
    # Reemplazos según diccionarios
    for d in (seniority_dict, modalidad_dict, puesto_dict):
        for k, v in d.items():
            s = re.sub(rf"\b{re.escape(k)}\b", v, s)
    
    # Eliminar palabras ruido
    for palabra in palabras_ruido:
        s = re.sub(rf"\b{re.escape(palabra)}\b", " ", s)
    
    # Eliminar duplicados y espacios
    s = re.sub(r"\s+", " ", s).strip()
    palabras = s.split()
    s = " ".join(sorted(set(palabras), key=palabras.index))
    return s


# ========================================
# 7️⃣ APLICAR LIMPIEZA
# ========================================
df["titulo_es_clean"] = df["titulo_es"].map(limpiar_titulo_total)

# ========================================
# 8️⃣ FRECUENCIA DE PALABRAS (para revisar)
# ========================================
todas_palabras = " ".join(df["titulo_es_clean"].dropna()).split()
freq = Counter(todas_palabras)
print("Palabras más frecuentes (posibles ruidos aún):")
for palabra, cnt in freq.most_common(30):
    print(f"{palabra:20s} -> {cnt}")

# ========================================
# 9️⃣ VISTA DE EJEMPLOS
# ========================================
muestra = df.dropna(subset=["titulo_es", "titulo_es_clean"]).sample(
    n=min(20, len(df)), random_state=42
)
pd.set_option("display.max_colwidth", 120)
print("\nEjemplo de limpieza completa:\n")
print(muestra[["titulo_es", "titulo_es_clean"]])

# ========================================
# 🔟 GUARDAR RESULTADO
# ========================================
print(df.head())


TypeError: can only concatenate list (not "dict") to list

In [ ]:
import re
import pandas as pd
import os
from collections import Counter

# === CARGAR DATASET ===


# ========================================
# 1️⃣ DICCIONARIO DE SENIORITY / NIVEL
# ========================================
seniority_dict = {
    "jr": "junior", "jr.": "junior", "junior": "junior", "joven": "junior",
    "ssr": "semisenior", "semi senior": "semisenior", "semi-senior": "semisenior",
    "sr": "senior", "sr.": "senior", "senior": "senior", "avanzado": "senior",
    "lead": "senior", "líder": "senior", "lider": "senior", "principal": "senior", 
    "experto": "senior", "manager": "senior", "coordinador": "senior", 
    "supervisor": "senior", "jefe": "senior", "gerente": "senior", 
    "director": "senior", "regente": "senior", "encargado": "senior",
    "responsable": "senior", "capitan": "senior", "gestor": "senior",
    "subgerente": "senior", "ejecutivo": "senior"
    
    
}

# ========================================
# 2️⃣ DICCIONARIO DE MODALIDAD / JORNADA
# ========================================
modalidad_dict = {
    "remoto": "remoto", "remote": "remoto", "teletrabajo": "remoto", "casa": "remoto",
    "híbrido": "hibrido", "hibrido": "hibrido", "hybrid": "hibrido",
    "presencial": "presencial", "onsite": "presencial", "oficina": "presencial",
    "full time": "tiempo completo", "jornada completa": "tiempo completo", 
    "tiempo completo": "tiempo completo", "completo": "tiempo completo",
    "part time": "medio tiempo", "medio tiempo": "medio tiempo", 
    "tiempo parcial": "medio tiempo", "parcial": "medio tiempo",
    "jornada diurna": "diurna", "diurno": "diurna",
    "jornada nocturna": "nocturna", "nocturno": "nocturna",
    "temporal": "temporal", "por horas": "temporal", "contrato": "temporal",
    "definido": "temporal", "freelance": "freelance", 
    "practicante": "intern", "internship": "intern", "pasante": "intern",
    "rotativo": "rotativo", "rotativa": "rotativo"
}

# ========================================
# 3️⃣ DICCIONARIO DE SINÓNIMOS DE PUESTOS
# ========================================
# ========================================
# 3️⃣ DICCIONARIO DE SINÓNIMOS DE PUESTOS - CORREGIDO
# ========================================
puesto_dict = {
    # Desarrollo de Software
    "desarrollador": "developer", "programador": "developer",
    "backend developer": "backend", "frontend developer": "frontend",
    "fullstack developer": "fullstack", "full stack": "fullstack",
    "software engineer": "ingeniero software", "ingeniero de software": "ingeniero software",
    "devops": "devops", "desarrollo": "developer",
    
    # Datos e IA
    "data engineer": "ingeniero datos", "ml engineer": "ingeniero ml",
    "ai engineer": "ingeniero ia", "analista": "analista",
    "data analyst": "analista datos", "analista de datos": "analista datos",
    "cientifico de datos": "data scientist", "data scientist": "data scientist",
    "bi": "business intelligence", "business intelligence": "business intelligence",
    
    # Soporte e Infraestructura
    "soporte técnico": "soporte", "técnico en soporte": "soporte",
    "soporte informático": "soporte", "soporte": "soporte",
    "network engineer": "ingeniero redes", "infraestructura": "infraestructura",
    "administrador": "administrador", "administrador de sistemas": "administrador sistemas",
    
    # Ingenierías - CORREGIDO
    "ingeniero": "ingeniero", "ingeniera": "ingeniero",
    "ing": "ingeniero",  # Si quieres incluir la abreviatura
    "ingeniero civil": "ingeniero civil", "civil": "ingeniero civil",
    "ingeniero eléctrico": "ingeniero electrico", "electrico": "ingeniero electrico",
    "ingeniero electromecánico": "ingeniero electromecanico", "electromecanico": "ingeniero electromecanico",
    "ingeniero industrial": "ingeniero industrial", "industrial": "ingeniero industrial",
    "ingeniero mecánico": "ingeniero mecanico", "mecanico": "ingeniero mecanico",
    "ingeniero de calidad": "ingeniero calidad", "calidad": "ingeniero calidad",
    "ingeniero de proyectos": "ingeniero proyectos", "proyectos": "ingeniero proyectos",
    
    # Diseño
    "diseñador": "diseñador", "diseñadora": "diseñador",
    "ux designer": "diseñador ux", "ui designer": "diseñador ui",
    "ux/ui": "diseñador uxui", "diseñador gráfico": "diseñador grafico",
    
    # Gestión
    "project manager": "gerente proyecto", "scrum master": "scrum master",
    "coordinador de proyecto": "gerente proyecto", "product owner": "product owner", 
    "product manager": "product manager",
    
    # Técnicos
    "técnico": "tecnico", "tecnologo": "tecnico", "technician": "tecnico",
    "mecánico": "mecanico", "electricista": "electricista",
    
    # Comercial/Ventas
    "vendedor": "vendedor", "comercial": "vendedor", "sales": "vendedor",
    "ejecutivo": "ejecutivo", "asesor": "asesor",
    
    # Especializados
    "farmacéutico": "farmaceutico", "microbiólogo": "microbiologo",
    "agrónomo": "agronomo", "veterinario": "veterinario"
}

# ========================================
# 4️⃣ LISTA DE UBICACIONES (Costa Rica)
# ========================================
ubicaciones_cr = [
    "costa rica", "san jose", "alajuela", "cartago", "heredia",
    "guanacaste", "puntarenas", "limon", "limón",
    "curridabat", "desamparados", "escazu", "escazú", "moravia", "tibas", "tibás",
    "santa ana", "belen", "belén", "barva", "santo domingo", "naranjo",
    "palmares", "upala", "nicoya", "liberia", "bagaces", "carrillo",
    "esparza", "osa", "corredores", "pococi", "pococí", "siquirres", 
    "pozos", "goicoechea", "sabanilla", "coronado", "san pablo",
    "san rafael", "san isidro", "santa domingo", "la union", "la unión",
    "san sebastian", "san sebastián", "purral", "lagunilla", "san antonio",
    "san luis", "san vicente", "san miguel", "san francisco", "san juan",
    "san pedro", "san marcos", "san ramon", "san ramón"
]

# ========================================
# 5️⃣ PALABRAS DE RUIDO
# ========================================
palabras_ruido = [
    # Licencias y requisitos de transporte
    "licencia", "b1", "b2", "b3", "al dia", "al día", "vigente",
    "montacargas", "vehiculo propio", "vehículo propio", "moto", "automóvil",
    
    # Disponibilidad
    "disponibilidad inmediata", "disponibilidad para viajar", "disponibilidad",
    "giras", "trasladarse", "movilizarse",
    
    # Experiencia
    "experiencia comprobada", "experiencia deseable", "experiencia",
    "años de experiencia", "anos de experiencia", "minimo", "mínimo",
    
    # Horarios
    "jornada", "turno", "diurno", "nocturno", "rotativo", "horario",
    
    # Entornos laborales
    "produccion", "producción", "planta", "operario", "operativa",
    "empresa", "contrato", "indefinido", "temporal", "corporativo",
    
    # Verbos de solicitud
    "se requiere", "requerimos", "solicitamos", "buscamos", "necesitamos",
    
    # Educación
    "bachillerato", "universitario", "licenciatura",
    "estudiante", "graduado", "titulo", "título",
    
    # Ubicaciones y referencias espaciales
    "sector", "zona", "ubicado en", "ubicada en", "oficina", "supermercado", 
    "bodega", "tienda", "local", "centro", "alrededores",
    
    # Palabras vacías
    "a", "o", "en", "para", "de", "con", "y", "el", "la", "los", "las", 
    "urgente", "importante", "nuevo", "nueva", "iii", "i","policy" , "asuntos", "publicos", "proyectos","ingles","portuges",
] + ubicaciones_cr 

# ========================================
# 6️⃣ FUNCIÓN DE LIMPIEZA TOTAL OPTIMIZADA
# ========================================
def limpiar_titulo_total(titulo):
    if pd.isna(titulo):
        return ""
    s = str(titulo).lower().strip()
    
    # Normalizar tildes
    s = (s.replace("á", "a").replace("é", "e")
           .replace("í", "i").replace("ó", "o")
           .replace("ú", "u").replace("ñ", "n"))
    
    # Eliminar símbolos raros
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s)
    
    # Lista de TODOS los términos a eliminar (seniority + modalidad + ruido)
    todos_eliminar = (
        list(set(seniority_dict.values())) + 
        list(set(modalidad_dict.values())) + 
        palabras_ruido
    )
    
    # PRIMERO: Normalizar todos los términos (seniority, modalidad, puestos)
    for d in [seniority_dict, modalidad_dict, puesto_dict]:
        for term, normalized in d.items():
            s = re.sub(rf"\b{re.escape(term)}\b", normalized, s)
    
    # SEGUNDO: Eliminar TODOS los términos no deseados
    for palabra in todos_eliminar:
        s = re.sub(rf"\b{re.escape(palabra)}\b", " ", s)
    
    # Limpiar resultado final
    s = re.sub(r"\s+", " ", s).strip()
    palabras = s.split()
    s = " ".join(sorted(set(palabras), key=palabras.index))
    
    return s

# ========================================
# 7️⃣ APLICAR LIMPIEZA
# ========================================
df["titulo_es_clean"] = df["titulo_es"].map(limpiar_titulo_total)

# ========================================
# 8️⃣ FRECUENCIA DE PALABRAS (para revisar)
# ========================================
todas_palabras = " ".join(df["titulo_es_clean"].dropna()).split()
freq = Counter(todas_palabras)
print("Palabras más frecuentes (posibles ruidos aún):")
for palabra, cnt in freq.most_common(30):
    print(f"{palabra:20s} -> {cnt}")

# ========================================
# 9️⃣ VISTA DE EJEMPLOS
# ========================================
muestra = df.dropna(subset=["titulo_es", "titulo_es_clean"]).sample(
    n=min(20, len(df)), random_state=42
)
pd.set_option("display.max_colwidth", 120)
print("\nEjemplo de limpieza completa:\n")
print(muestra[["titulo_es", "titulo_es_clean"]])

# ========================================
# 🔟 GUARDAR RESULTADO
# ========================================
print(df.head())

Palabras más frecuentes (posibles ruidos aún):
ingeniero            -> 90
tecnico              -> 72
mantenimiento        -> 28
vendedor             -> 26
electromecanico      -> 26
ventas               -> 20
analista             -> 17
developer            -> 17
farmaceutico         -> 17
especialista         -> 15
industrial           -> 13
mecanico             -> 12
asesor               -> 12
asistente            -> 12
profesor             -> 11
personal             -> 11
soporte              -> 10
medico               -> 9
construccion         -> 9
seguridad            -> 8
docente              -> 8
auxiliar             -> 8
cocinero             -> 8
ingenieria           -> 7
chofer               -> 7
agente               -> 7
electrico            -> 6
datos                -> 6
pasantia             -> 6
arquitecto           -> 6

Ejemplo de limpieza completa:

                                                              titulo_es  \
1066                                             

In [ ]:
# ========================================
#  Prueba de titulos 
# ========================================
# --- Vista de prueba: comparar títulos originales vs títulos limpios (sin NaN) ---

# Filtrar filas que no sean NaN en ambas columnas
df_validos = df.dropna(subset=["titulo_es", "titulo_es_clean"])

# Tomar una muestra aleatoria de hasta 20 filas
muestra = df_validos[["titulo_es", "titulo_es_clean"]].sample(
    n=min(20, len(df_validos)),
    random_state=42
)

# Mostrar en pantalla
import pandas as pd
pd.set_option("display.max_colwidth", 100)
display(muestra)

# (Opcional) Guardar la muestra para revisar manualmente
print("Archivo 'muestra_titulos_limpios.csv' guardado exitosamente.")


,titulo_es,titulo_es_clean
1066,Arquitecto / Ingeniero,arquitecto ingeniero
98,tecnico electromecanico / electronico /electrotecnia (urgente),tecnico ingeniero electromecanico electronico electrotecnia
152,ingeniero/a consultor de calidad,ingeniero consultor calidad
958,Asistente en Gestión Ambiental,asistente gestion ambiental
1218,Operario/a de producción con licencia de montacargas al día,
1222,(Turno Nocturno) Electricista de Mantenimiento Industrial III,electricista mantenimiento ingeniero industrial
1117,Ingeniero(a) Civil / Construcción,ingeniero civil construccion
102,tecnico en mantenimiento,tecnico mantenimiento
1340,Policy Manager – Asuntos Públicos y Relacionamiento Gubernamental,relacionamiento gubernamental
1418,Jefe de Ventas Farmacéutica,ventas farmaceutica


Archivo 'muestra_titulos_limpios.csv' guardado exitosamente.
